In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

---
# **Read data**

In [ ]:
train = pd.read_csv('/kaggle/input/unsw-nb15/UNSW_NB15_testing-set.csv', sep=',', encoding='cp1252')
test = pd.read_csv('/kaggle/input/unsw-nb15/UNSW_NB15_training-set.csv', sep=',', encoding='cp1252')
train.shape, test.shape

## **Concatenate test and train set**

In [ ]:
df = pd.concat([train, test])
df.shape

---
# **Missing values**

In [ ]:
pd.set_option('display.max_columns', None)
pd.DataFrame(df.isna().sum().to_dict(), index=[0])

---
# **Columns**

In [ ]:
df.columns

---
# **label**

In [ ]:
df.label.value_counts().plot.bar()

In [ ]:
df.label.value_counts(normalize=True).plot.bar()

In [ ]:
df.label.value_counts(), df.label.value_counts(normalize=True)

---
# **attack_cat**

In [ ]:
df.attack_cat.value_counts().plot.bar()

---
# **cols types**

In [ ]:
df.dtypes.value_counts()

---
## **Cat features**

In [ ]:
cat_features = df.columns[df.dtypes == 'object']
cat_features

In [ ]:
for i in cat_features:
    if i == 'proto':
        print(i, '\n', df[i].value_counts().to_dict())
    else :
        plt.figure()
        df[i].value_counts().plot.bar()

---
# **Num features**

In [ ]:
num_features = df.columns[df.dtypes != 'object']
num_features

In [ ]:
import seaborn as sns
import warnings 
warnings.filterwarnings('ignore')


for i in num_features:
    plt.figure()
    sns.distplot(df[i])

---
# **Correlation**

In [ ]:
corr = df.corr()

In [ ]:
import seaborn as sns

sns.set_theme('notebook')

mask = np.triu(np.ones_like(corr, dtype=bool))
cmap = sns.diverging_palette(230, 20, as_cmap=True)

plt.figure(figsize=(20,12))
sns.heatmap(corr, mask=mask, cmap=cmap, center=0, square=True, linewidths=1.)

---
# **PIPELINES**

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OneHotEncoder
from sklearn.impute import SimpleImputer

In [ ]:
Y = df.label
X = df.drop(['label'], axis=1)

In [ ]:
cat_features = X.columns[X.dtypes == 'object']
num_features = X.columns[X.dtypes != 'object']

In [ ]:
num_pipeline = make_pipeline(SimpleImputer(),
                            StandardScaler())

cat_pipeline = make_pipeline(SimpleImputer(strategy='most_frequent'),
                            OneHotEncoder(handle_unknown='ignore'))

In [ ]:
preprocessor = make_column_transformer((num_pipeline, num_features),
                                      (cat_pipeline, cat_features))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc = make_pipeline(preprocessor, RandomForestClassifier(n_estimators=500, n_jobs=10, bootstrap=True, random_state=42))

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, shuffle=True)

In [ ]:
model = rfc.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import classification_report

ypred = model.predict(X_test)
print(classification_report(y_test, ypred))

> ----
## dropping some features

In [ ]:
Y = df.label
X = df.drop(['label', 'ï»¿id', 'attack_cat'], axis=1)

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X,Y, test_size=0.2, shuffle=True)

In [ ]:
cat_features = X.columns[X.dtypes == 'object']
num_features = X.columns[X.dtypes != 'object']

In [ ]:
num_pipeline = make_pipeline(SimpleImputer(),
                            StandardScaler())

cat_pipeline = make_pipeline(SimpleImputer(strategy='most_frequent'),
                            OneHotEncoder(handle_unknown='ignore'))

In [ ]:
preprocessor = make_column_transformer((num_pipeline, num_features),
                                      (cat_pipeline, cat_features))

In [ ]:
from sklearn.ensemble import RandomForestClassifier

rfc2 = make_pipeline(preprocessor, RandomForestClassifier(n_estimators=500, n_jobs=-1, bootstrap=True, random_state=42))

In [ ]:
model2 = rfc2.fit(X_train, y_train)

from sklearn.metrics import classification_report

ypred2 = model2.predict(X_test)
print(classification_report(y_test, ypred2))